# 가중치 기법

In [2]:
collection = [
    ("Document1", "This is a sample"),  # a 가 중요
    ("Document2","This is another sample"), # another 가 중요
    ("Document2","This is not sample") # not 이 중요
]

query = "this is a sample"

In [3]:
# 전역변수 만들기
# in-memory (Hash-key값)
# {단어1 : 포스팅위치, 단어2: 포스팅위치, ...}
globalLexicon = dict()
# [0:문서, 1:문서2, ...]
globalDocument = list()

# disk  ( out memory )
# [0:(단어 idx, 문서 idx, 빈도, 다음주소), ...]
# 메모리에 존재안하고 파일에만 존재해도 됨, 위치를 알고 있어서
globalPosting = list()

In [50]:
docIdx = len(globalDocument)
docIdx

6

In [4]:
for (docName, docContent) in collection:
    # Pointer 대체용, Key, Document이름은 절대로 겹치지 않는다는 가정
    docIdx = len(globalDocument)
    globalDocument.append(docName)
    
    # {단어idx:빈도, 단어idx:빈도, ...}
    localPosting = dict()
    
    # 로컬 / 띄어쓰기 단위로 
    for term in docContent.lower().split():   
        if term not in localPosting.keys():
            localPosting[term] = 1
        else:
            localPosting[term] += 1
    
    # fp -> struct(단어,빈도) ( localPosting)
    # Merge와 sorting이 같이 있는 것 
    for indexTerm, termFreq in localPosting.items():
        if indexTerm not in globalLexicon.keys():
            lexiconIdx = len(globalLexicon)
            postingIdx = len(globalPosting) # fseek
            postingData = (lexiconIdx,docIdx,termFreq, -1)
            globalPosting.append(postingData)
            globalLexicon[indexTerm] =  postingIdx   # globalPosting 위치 (ptr:idx)
        else:
            lexiconIdx = list(globalLexicon.keys()).index(indexTerm)
            postingIdx =  len(globalPosting)
            beforeIdx = globalLexicon[indexTerm]
            postingData = (lexiconIdx,docIdx,termFreq, beforeIdx)
            globalPosting.append(postingData)
            globalLexicon[indexTerm] =  postingIdx   # globalPosting 위치 (ptr:idx)
#     print(localPosting)
# print(globalDocument)  
#         if term not in globalLexicon.keys():
#             localPosting
#             lexiconIdx = len(globalLexicon)   # 처음엔 Length가 0일 것. 
            
            

In [5]:
len(globalDocument)

3

In [6]:
globalDocument.index("Document1")

0

In [7]:
globalLexicon

{'this': 8, 'is': 9, 'a': 2, 'sample': 11, 'another': 6, 'not': 10}

In [8]:
globalPosting
# 끝이 -1 이면 끝 
# 세번째 인덱스가 빈도 

[(0, 0, 1, -1),
 (1, 0, 1, -1),
 (2, 0, 1, -1),
 (3, 0, 1, -1),
 (0, 1, 1, 0),
 (1, 1, 1, 1),
 (4, 1, 1, -1),
 (3, 1, 1, 3),
 (0, 2, 1, 4),
 (1, 2, 1, 5),
 (5, 2, 1, -1),
 (3, 2, 1, 7)]

In [9]:
for indexTerm, postingIdx in globalLexicon.items():
    # indexTerm : 단어,  postingIdx : 위치, 
    print(indexTerm)
    
    while True: # Postrin Next : -1 -> -1이 될 때 까지 찾으려고
        if postingIdx == -1:
            break
            
        postingData = globalPosting[postingIdx]
        print( " {0} / {1} / {2} ".format(globalDocument[postingData[1]],postingData[2],postingData[3]))
        postingIdx = postingData[3]

this
 Document2 / 1 / 4 
 Document2 / 1 / 0 
 Document1 / 1 / -1 
is
 Document2 / 1 / 5 
 Document2 / 1 / 1 
 Document1 / 1 / -1 
a
 Document1 / 1 / -1 
sample
 Document2 / 1 / 7 
 Document2 / 1 / 3 
 Document1 / 1 / -1 
another
 Document2 / 1 / -1 
not
 Document2 / 1 / -1 


# 가중치 기법

http://www.cs.virginia.edu/%7Ehw5x/Course/IR2015/_site/docs/PDFs/Boolean&VS%20model.pdf

TF : Document 안에서 얼마나 중요한지
DF : Collection 안에서 얼마나 중요한지 / 많이 나타난 정도

# TF

In [10]:
from math import log10

def rawTF(freq):
    return freq

def normTF(freq,totalCount):
    return (freq / totalCount)

def logTF(freq):
    if freq > 0:
        return 1 + log10(freq)
    else:
        return 0

def maxTF(a,freq,maxFreq):   # double normalization K -  doc : 0 / query : 0.5
    return a + ((1-a)* (freq/maxFreq))

In [11]:
globalTF = list()

for (docName, docContent) in collection:
    localPosting = dict()
    maxCount = 0
    # 로컬 / 띄어쓰기 단위로 
    for term in docContent.lower().split():   
        maxCount += 1
        if term not in localPosting.keys():
            localPosting[term] = 1
        else:
            localPosting[term] += 1
            
    print(docName)
    a = 0.5
    maxFreq = max(localPosting.values())
    
    for term,freq in localPosting.items():
        print("1. {0} rawTF : {1}".format(term,rawTF(freq)))
        print("2. {0} normTF : {1}".format(term,normTF(freq,maxCount)))
        print("3. {0} logTF : {1}".format(term,logTF(freq)))
        print("4. {0} maxTF : {1}".format(term,maxTF(a,freq,maxFreq)))
        print()
        localPosting[term] = maxTF(a,freq,maxFreq)
        
    for indexTerm, termTF in localPosting.items():
        if indexTerm not in globalLexicon.keys():
            lexiconIdx = len(globalLexicon)
            postingIdx = len(globalTF) # fseek
            postingData = (lexiconIdx,docIdx,termTF, -1)
            globalTF.append(postingData)
            globalLexicon[indexTerm] =  postingIdx   # globalPosting 위치 (ptr:idx)
        else:
            lexiconIdx = list(globalLexicon.keys()).index(indexTerm)
            postingIdx =  len(globalTF)
            beforeIdx = globalLexicon[indexTerm]
            postingData = (lexiconIdx,docIdx,termTF, beforeIdx)
            globalTF.append(postingData)
            globalLexicon[indexTerm] =  postingIdx   

Document1
1. this rawTF : 1
2. this normTF : 0.25
3. this logTF : 1.0
4. this maxTF : 1.0

1. is rawTF : 1
2. is normTF : 0.25
3. is logTF : 1.0
4. is maxTF : 1.0

1. a rawTF : 1
2. a normTF : 0.25
3. a logTF : 1.0
4. a maxTF : 1.0

1. sample rawTF : 1
2. sample normTF : 0.25
3. sample logTF : 1.0
4. sample maxTF : 1.0

Document2
1. this rawTF : 1
2. this normTF : 0.25
3. this logTF : 1.0
4. this maxTF : 1.0

1. is rawTF : 1
2. is normTF : 0.25
3. is logTF : 1.0
4. is maxTF : 1.0

1. another rawTF : 1
2. another normTF : 0.25
3. another logTF : 1.0
4. another maxTF : 1.0

1. sample rawTF : 1
2. sample normTF : 0.25
3. sample logTF : 1.0
4. sample maxTF : 1.0

Document2
1. this rawTF : 1
2. this normTF : 0.25
3. this logTF : 1.0
4. this maxTF : 1.0

1. is rawTF : 1
2. is normTF : 0.25
3. is logTF : 1.0
4. is maxTF : 1.0

1. not rawTF : 1
2. not normTF : 0.25
3. not logTF : 1.0
4. not maxTF : 1.0

1. sample rawTF : 1
2. sample normTF : 0.25
3. sample logTF : 1.0
4. sample maxTF : 1.0



In [12]:
print(globalPosting), print(globalTF)

[(0, 0, 1, -1), (1, 0, 1, -1), (2, 0, 1, -1), (3, 0, 1, -1), (0, 1, 1, 0), (1, 1, 1, 1), (4, 1, 1, -1), (3, 1, 1, 3), (0, 2, 1, 4), (1, 2, 1, 5), (5, 2, 1, -1), (3, 2, 1, 7)]
[(0, 2, 1.0, 8), (1, 2, 1.0, 9), (2, 2, 1.0, 2), (3, 2, 1.0, 11), (0, 2, 1.0, 0), (1, 2, 1.0, 1), (4, 2, 1.0, 6), (3, 2, 1.0, 3), (0, 2, 1.0, 4), (1, 2, 1.0, 5), (5, 2, 1.0, 10), (3, 2, 1.0, 7)]


(None, None)

#### 0~1 사이의 값을 가지는 "maxTF" 가 상대적인 중요도를 알기에 좋다.

# IDF 

### 적게 나오는 단어에 대해 더 높은 가중치를 부여하자!

tf-idf  : 
https://en.wikipedia.org/wiki/Tf–idf



In [13]:
# 일반적인 IDF
def rawIdf(df, N):
    return log10(N / df)

# the,a, 불용어 안날림 => to be or not to be 
def smoothingIdf(df,N):
    return log10((N+1) / df)

def probabilityIdf(df,N):
    return log10((N-df+1) / df)

### maxTF , rawIDF 를 일반적으로 사용

In [14]:
collection = [
    ("Document1", "This is a sample"),  # a 가 중요
    ("Document2","This is another sample"), # another 가 중요
    ("Document3","This is not sample"), # not 이 중요
]

query = "this is sample"

In [15]:
globalLexicon = dict()
globalDocument = list()
globalPosting = list()

for (docName, docContent) in collection:
    # Pointer 대체용, Key, Document이름은 절대로 겹치지 않는다는 가정
    docIdx = len(globalDocument)
    globalDocument.append(docName)
    
    # {단어idx:빈도, 단어idx:빈도, ...}
    localPosting = dict()
    
    # 로컬 / 띄어쓰기 단위로 
    for term in docContent.lower().split():   
        if term not in localPosting.keys():
            localPosting[term] = 1
        else:
            localPosting[term] += 1
    
    maxFreq = max(localPosting.values())
    
    # fp -> struct(단어,빈도) ( localPosting)
    # Merge와 sorting이 같이 있는 것 
    for indexTerm, termFreq in localPosting.items():
        if indexTerm not in globalLexicon.keys():
            lexiconIdx = len(globalLexicon)
            postingIdx = len(globalPosting) # fseek
            postingData = [lexiconIdx,docIdx,maxTF(0,termFreq,maxFreq), -1]
            globalPosting.append(postingData)
            globalLexicon[indexTerm] =  postingIdx   # globalPosting 위치 (ptr:idx)
        else:
            lexiconIdx = list(globalLexicon.keys()).index(indexTerm)
            postingIdx =  len(globalPosting)
            beforeIdx = globalLexicon[indexTerm]
            postingData = [lexiconIdx,docIdx,maxTF(0,termFreq,maxFreq), beforeIdx]
            globalPosting.append(postingData)
            globalLexicon[indexTerm] =  postingIdx   # globalPosting 위치 (ptr:idx)
#     print(localPosting)
# print(globalDocument)  
#         if term not in globalLexicon.keys():
#             localPosting
#             lexiconIdx = len(globalLexicon)   # 처음엔 Length가 0일 것. 
            
            

In [16]:
globalPosting
# 가중치가 다 1.0 

[[0, 0, 1.0, -1],
 [1, 0, 1.0, -1],
 [2, 0, 1.0, -1],
 [3, 0, 1.0, -1],
 [0, 1, 1.0, 0],
 [1, 1, 1.0, 1],
 [4, 1, 1.0, -1],
 [3, 1, 1.0, 3],
 [0, 2, 1.0, 4],
 [1, 2, 1.0, 5],
 [5, 2, 1.0, -1],
 [3, 2, 1.0, 7]]

In [17]:
# 최종 코드 ( TF & IDF )
N = len(globalDocument)
globalLexiconIDF = dict()

for indexTerm, postingIdx in globalLexicon.items():
    df = 0
    oldPostingIdx = postingIdx
    
    while True: 
        if postingIdx == -1:
            break
            
        df += 1
        postingData = globalPosting[postingIdx]
        postingIdx = postingData[3]
        
    postingIdx = oldPostingIdx
    idf = rawIdf(df,N)
    globalLexiconIDF[indexTerm] = idf
    
    print("{0} / IDF-{1}".format(indexTerm,idf))
    
    while True: 
        if postingIdx == -1:
            break
         
        postingData = globalPosting[postingIdx]
        TF = postingData[2]
        postingData[2] = postingData[2] * idf
        print(" Document:{0} / TF:{1} / TF-IDF:{2}".format(globalDocument[postingData[1]],
                                                         TF,
                                                          globalPosting[postingIdx][2]))
        postingIdx = postingData[3]
        

this / IDF-0.0
 Document:Document3 / TF:1.0 / TF-IDF:0.0
 Document:Document2 / TF:1.0 / TF-IDF:0.0
 Document:Document1 / TF:1.0 / TF-IDF:0.0
is / IDF-0.0
 Document:Document3 / TF:1.0 / TF-IDF:0.0
 Document:Document2 / TF:1.0 / TF-IDF:0.0
 Document:Document1 / TF:1.0 / TF-IDF:0.0
a / IDF-0.47712125471966244
 Document:Document1 / TF:1.0 / TF-IDF:0.47712125471966244
sample / IDF-0.0
 Document:Document3 / TF:1.0 / TF-IDF:0.0
 Document:Document2 / TF:1.0 / TF-IDF:0.0
 Document:Document1 / TF:1.0 / TF-IDF:0.0
another / IDF-0.47712125471966244
 Document:Document2 / TF:1.0 / TF-IDF:0.47712125471966244
not / IDF-0.47712125471966244
 Document:Document3 / TF:1.0 / TF-IDF:0.47712125471966244


In [51]:
globalPosting

[[0, 0, 0.13540691451574405, -1],
 [1, 0, 0.13540691451574405, -1],
 [2, 0, 0.29601003688313354, -1],
 [3, 0, 0.0213534028111804, -1],
 [0, 1, 0.13540691451574405, 0],
 [1, 1, 0.13540691451574405, 1],
 [4, 1, 0.7141906972359384, -1],
 [3, 1, 0.0213534028111804, 3],
 [0, 2, 0.13540691451574405, 4],
 [1, 2, 0.13540691451574405, 5],
 [5, 2, 0.05906749310924163, -1],
 [3, 2, 0.0213534028111804, 7],
 [2, 3, 0.29601003688313354, 2],
 [5, 3, 0.05906749310924163, 10],
 [3, 3, 0.0213534028111804, 11],
 [5, 4, 0.05906749310924163, 13],
 [5, 5, 0.05906749310924163, 15],
 [3, 5, 0.0213534028111804, 14]]

In [18]:
def euclidean(x,y):
    return (x - y) ** 2

In [19]:
query # Document
queryPosting = dict()

for term in query.lower().split():   
        if term not in queryPosting.keys():
            queryPosting[term] = 1
        else:
            queryPosting[term] += 1
    
maxFreq = max(queryPosting.values())
    
    # fp -> struct(단어,빈도) ( localPosting)
    # Merge와 sorting이 같이 있는 것 
for indexTerm, termFreq in queryPosting.items():
    queryPosting[indexTerm] = maxTF(0.5,termFreq,maxFreq)

In [20]:
queryPosting

{'this': 1.0, 'is': 1.0, 'sample': 1.0}

In [21]:
globalLexicon.items()

dict_items([('this', 8), ('is', 9), ('a', 2), ('sample', 11), ('another', 6), ('not', 10)])

In [22]:
candidateList = dict()   # 아직까진 검색후보군이어서 

for indexTerm, postingIdx in globalLexicon.items():
    queryTFIDF = 0
    
    if indexTerm in queryPosting.keys():
        queryTFIDF = queryPosting[indexTerm] * globalLexiconIDF[indexTerm]


    while True: 
        if postingIdx == -1:
            break

        postingData = globalPosting[postingIdx]
        postingIdx = postingData[3]
        documentWeight = postingData[2]

        if postingData[1] not in candidateList.keys():
            candidateList[postingData[1]] = euclidean(queryTFIDF,documentWeight) # 각 다큐먼트마다 누적시켜야함
        else:
             candidateList[postingData[1]] += euclidean(queryTFIDF,documentWeight) 


In [23]:
resultList = sorted(candidateList.items(), key=lambda x:x[1])

for i, (documentIdx, distance) in  enumerate(resultList):
    print("순위: {0}. 문서 : {1} / Distance:{2}".format((i+1),globalDocument[documentIdx], distance))

순위: 1. 문서 : Document3 / Distance:0.227644691705265
순위: 2. 문서 : Document2 / Distance:0.227644691705265
순위: 3. 문서 : Document1 / Distance:0.227644691705265


### query 바꿔서


In [24]:
query = "this is a sample"

queryPosting = dict()

for term in query.lower().split():   
        if term not in queryPosting.keys():
            queryPosting[term] = 1
        else:
            queryPosting[term] += 1
    
maxFreq = max(queryPosting.values())
    
    # fp -> struct(단어,빈도) ( localPosting)
    # Merge와 sorting이 같이 있는 것 
for indexTerm, termFreq in queryPosting.items():
    queryPosting[indexTerm] = maxTF(0.5,termFreq,maxFreq)
    
candidateList = dict()   # 아직까진 검색후보군이어서 

for indexTerm, postingIdx in globalLexicon.items():
    queryTFIDF = 0
    
    if indexTerm in queryPosting.keys():
        queryTFIDF = queryPosting[indexTerm] * globalLexiconIDF[indexTerm]


    while True: 
        if postingIdx == -1:
            break

        postingData = globalPosting[postingIdx]
        postingIdx = postingData[3]
        documentWeight = postingData[2]

        if postingData[1] not in candidateList.keys():
            candidateList[postingData[1]] = euclidean(queryTFIDF,documentWeight) # 각 다큐먼트마다 누적시켜야함
        else:
             candidateList[postingData[1]] += euclidean(queryTFIDF,documentWeight) 

resultList = sorted(candidateList.items(), key=lambda x:x[1])

for i, (documentIdx, distance) in  enumerate(resultList):
    print("순위: {0}. 문서 : {1} / Distance:{2}".format((i+1),globalDocument[documentIdx], distance))
    print(" {0}".format(collection[documentIdx][1]))
    
    # 거리가 0인것은 아예 같다는 것. 

순위: 1. 문서 : Document1 / Distance:0.0
 This is a sample
순위: 2. 문서 : Document3 / Distance:0.227644691705265
 This is not sample
순위: 3. 문서 : Document2 / Distance:0.227644691705265
 This is another sample


### Smoothing 사용

collection 도 변경. 

In [25]:
collection = [
    ("Document1", "This is a sample"),  # a 가 중요
    ("Document2","This is another sample"), # another 가 중요
    ("Document3","This is not sample"), # not 이 중요
    ("Document4","a not sample"),
    ("Document5","not"),
    ("Document5","not sample"),
]

query = "this is sample"

In [26]:
globalLexicon = dict()
globalDocument = list()
globalPosting = list()

for (docName, docContent) in collection:
    # Pointer 대체용, Key, Document이름은 절대로 겹치지 않는다는 가정
    docIdx = len(globalDocument)
    globalDocument.append(docName)
    
    # {단어idx:빈도, 단어idx:빈도, ...}
    localPosting = dict()
    
    # 로컬 / 띄어쓰기 단위로 
    for term in docContent.lower().split():   
        if term not in localPosting.keys():
            localPosting[term] = 1
        else:
            localPosting[term] += 1
    
    maxFreq = max(localPosting.values())
    
    # fp -> struct(단어,빈도) ( localPosting)
    # Merge와 sorting이 같이 있는 것 
    for indexTerm, termFreq in localPosting.items():
        if indexTerm not in globalLexicon.keys():
            lexiconIdx = len(globalLexicon)
            postingIdx = len(globalPosting) # fseek
            postingData = [lexiconIdx,docIdx,maxTF(0,termFreq,maxFreq), -1]
            globalPosting.append(postingData)
            globalLexicon[indexTerm] =  postingIdx   # globalPosting 위치 (ptr:idx)
        else:
            lexiconIdx = list(globalLexicon.keys()).index(indexTerm)
            postingIdx =  len(globalPosting)
            beforeIdx = globalLexicon[indexTerm]
            postingData = [lexiconIdx,docIdx,maxTF(0,termFreq,maxFreq), beforeIdx]
            globalPosting.append(postingData)
            globalLexicon[indexTerm] =  postingIdx   # globalPosting 위치 (ptr:idx)
#     print(localPosting)
# print(globalDocument)  
#         if term not in globalLexicon.keys():
#             localPosting
#             lexiconIdx = len(globalLexicon)   # 처음엔 Length가 0일 것. 
            
            

In [27]:
# 최종 코드 ( TF & IDF )
N = len(globalDocument)
globalLexiconIDF = dict()

for indexTerm, postingIdx in globalLexicon.items():
    df = 0
    oldPostingIdx = postingIdx
    
    while True: 
        if postingIdx == -1:
            break
            
        df += 1
        postingData = globalPosting[postingIdx]
        postingIdx = postingData[3]
        
    postingIdx = oldPostingIdx
    idf = smoothingIdf(df,N)
    globalLexiconIDF[indexTerm] = idf
    
    print("{0} / IDF-{1}".format(indexTerm,idf))
    
    while True: 
        if postingIdx == -1:
            break
         
        postingData = globalPosting[postingIdx]
        TF = postingData[2]
        postingData[2] = postingData[2] * idf
        print(" Document:{0} / TF:{1} / TF-IDF:{2}".format(globalDocument[postingData[1]],
                                                         TF,
                                                          globalPosting[postingIdx][2]))
        postingIdx = postingData[3]
        

this / IDF-0.36797678529459443
 Document:Document3 / TF:1.0 / TF-IDF:0.36797678529459443
 Document:Document2 / TF:1.0 / TF-IDF:0.36797678529459443
 Document:Document1 / TF:1.0 / TF-IDF:0.36797678529459443
is / IDF-0.36797678529459443
 Document:Document3 / TF:1.0 / TF-IDF:0.36797678529459443
 Document:Document2 / TF:1.0 / TF-IDF:0.36797678529459443
 Document:Document1 / TF:1.0 / TF-IDF:0.36797678529459443
a / IDF-0.5440680443502757
 Document:Document4 / TF:1.0 / TF-IDF:0.5440680443502757
 Document:Document1 / TF:1.0 / TF-IDF:0.5440680443502757
sample / IDF-0.146128035678238
 Document:Document5 / TF:1.0 / TF-IDF:0.146128035678238
 Document:Document4 / TF:1.0 / TF-IDF:0.146128035678238
 Document:Document3 / TF:1.0 / TF-IDF:0.146128035678238
 Document:Document2 / TF:1.0 / TF-IDF:0.146128035678238
 Document:Document1 / TF:1.0 / TF-IDF:0.146128035678238
another / IDF-0.8450980400142568
 Document:Document2 / TF:1.0 / TF-IDF:0.8450980400142568
not / IDF-0.24303804868629444
 Document:Document5 

In [28]:
query = "not"

queryPosting = dict()

for term in query.lower().split():   
        if term not in queryPosting.keys():
            queryPosting[term] = 1
        else:
            queryPosting[term] += 1
    
maxFreq = max(queryPosting.values())
    
    # fp -> struct(단어,빈도) ( localPosting)
    # Merge와 sorting이 같이 있는 것 
for indexTerm, termFreq in queryPosting.items():
    queryPosting[indexTerm] = maxTF(0.5,termFreq,maxFreq)
    
candidateList = dict()   # 아직까진 검색후보군이어서 

for indexTerm, postingIdx in globalLexicon.items():
    queryTFIDF = 0
    
    if indexTerm in queryPosting.keys():
        queryTFIDF = queryPosting[indexTerm] * globalLexiconIDF[indexTerm]


    while True: 
        if postingIdx == -1:
            break

        postingData = globalPosting[postingIdx]
        postingIdx = postingData[3]
        documentWeight = postingData[2]

        if postingData[1] not in candidateList.keys():
            candidateList[postingData[1]] = euclidean(queryTFIDF,documentWeight) # 각 다큐먼트마다 누적시켜야함
        else:
             candidateList[postingData[1]] += euclidean(queryTFIDF,documentWeight) 

resultList = sorted(candidateList.items(), key=lambda x:x[1])

print(query)
for i, (documentIdx, distance) in  enumerate(resultList):
    print("순위: {0}. 문서 : {1} / Distance:{2}".format((i+1),globalDocument[documentIdx], distance))
    print(" {0}".format(collection[documentIdx][1]))
    
    # 거리가 0인것은 아예 같다는 것. 

not
순위: 1. 문서 : Document5 / Distance:0.0
 not
순위: 2. 문서 : Document5 / Distance:0.0213534028111804
 not sample
순위: 3. 문서 : Document3 / Distance:0.2921672318426685
 This is not sample
순위: 4. 문서 : Document4 / Distance:0.31736343969431396
 a not sample
순위: 5. 문서 : Document1 / Distance:0.588177268725802
 This is a sample
순위: 6. 문서 : Document2 / Distance:1.0063579290786069
 This is another sample


# vector space cosine similarity
# 유사도

In [29]:
def innerProduct(x,y):
    return x * y

In [32]:
candidateList = dict()  
globalDocumentLength = dict()

for indexTerm, queryWeight in queryPosting.items():
    if indexTerm in globalLexicon.keys():
        postingIdx = globalLexicon[indexTerm]
        
        while True: 
            if postingIdx == -1:
                break 

            postingData = globalPosting[postingIdx]
            postingIdx = postingData[3]
            documentWeight = postingData[2]
            
            
            if postingData[1] not in candidateList.keys():
                candidateList[postingData[1]] = innerProduct(queryWeight,documentWeight) # 각 다큐먼트마다 누적시켜야함
            else:
                 candidateList[postingData[1]] += innerProduct(queryWeight,documentWeight) 
                    
for documentIdx, sumProduct in candidateList.items():
    candidateList[documentIdx] /= globalDocumentLength[documentIdx]

KeyError: 5

In [33]:
# 최종 코드 ( TF & IDF )
N = len(globalDocument)
globalLexiconIDF = dict()
globalDocumentLength = dict()

for indexTerm, postingIdx in globalLexicon.items():
    df = 0
    oldPostingIdx = postingIdx
    
    while True: 
        if postingIdx == -1:
            break
            
        df += 1
        postingData = globalPosting[postingIdx]
        postingIdx = postingData[3]
        
    postingIdx = oldPostingIdx
    idf = smoothingIdf(df,N)
    globalLexiconIDF[indexTerm] = idf
    
    print("{0} / IDF-{1}".format(indexTerm,idf))
    
    while True: 
        if postingIdx == -1:
            break
         
        postingData = globalPosting[postingIdx]
        TF = postingData[2]
        postingData[2] = postingData[2] * idf
        print(" Document:{0} / TF:{1} / TF-IDF:{2}".format(globalDocument[postingData[1]],
                                                         TF,
                                                          globalPosting[postingIdx][2]))
        postingIdx = postingData[3]
        
        if postingData[1] not in globalDocumentLength.keys():
                globalDocumentLength[postingData[1]] = globalPosting[postingIdx][2] ** 2 # 각 다큐먼트마다 누적시켜야함
        else:
                globalDocumentLength[postingData[1]] += globalPosting[postingIdx][2] ** 2

this / IDF-0.36797678529459443
 Document:Document3 / TF:0.36797678529459443 / TF-IDF:0.13540691451574405
 Document:Document2 / TF:0.36797678529459443 / TF-IDF:0.13540691451574405
 Document:Document1 / TF:0.36797678529459443 / TF-IDF:0.13540691451574405
is / IDF-0.36797678529459443
 Document:Document3 / TF:0.36797678529459443 / TF-IDF:0.13540691451574405
 Document:Document2 / TF:0.36797678529459443 / TF-IDF:0.13540691451574405
 Document:Document1 / TF:0.36797678529459443 / TF-IDF:0.13540691451574405
a / IDF-0.5440680443502757
 Document:Document4 / TF:0.5440680443502757 / TF-IDF:0.29601003688313354
 Document:Document1 / TF:0.5440680443502757 / TF-IDF:0.29601003688313354
sample / IDF-0.146128035678238
 Document:Document5 / TF:0.146128035678238 / TF-IDF:0.0213534028111804
 Document:Document4 / TF:0.146128035678238 / TF-IDF:0.0213534028111804
 Document:Document3 / TF:0.146128035678238 / TF-IDF:0.0213534028111804
 Document:Document2 / TF:0.146128035678238 / TF-IDF:0.0213534028111804
 Documen

In [34]:
resultList = sorted(candidateList.items(), key=lambda x:x[1], reverse=True)

print(query)
for i, (documentIdx, distance) in  enumerate(resultList):
    print("순위: {0}. 문서 : {1} / 유사도:{2}".format((i+1),globalDocument[documentIdx], distance))
    print(" {0}".format(collection[documentIdx][1]))
    
    # 거리가 0인것은 아예 같다는 것. 

not
순위: 1. 문서 : Document5 / 유사도:0.24303804868629444
 not sample
순위: 2. 문서 : Document5 / 유사도:0.24303804868629444
 not
순위: 3. 문서 : Document4 / 유사도:0.24303804868629444
 a not sample
순위: 4. 문서 : Document3 / 유사도:0.24303804868629444
 This is not sample


# + 색인어 목록 만들기

In [35]:
from konlpy.corpus import kobill

def getLexiconBySet():
    lexicon = list()
    
    for docName in kobill.fileids():
        document = kobill.open(docName).read()
        
        for token in document.split():
            lexicon.extend(token)
            
    return list(set(lexicon))
        

In [36]:
txt = getLexiconBySet()
txt

['상',
 '첫',
 '는',
 '<',
 '〈',
 '책',
 '식',
 '순',
 '측',
 '견',
 '심',
 '모',
 '감',
 '育',
 '록',
 '‘',
 '담',
 '퇴',
 '때',
 't',
 '갑',
 '자',
 '0',
 '날',
 '탑',
 '취',
 '갖',
 '려',
 '분',
 '주',
 '.',
 '9',
 '조',
 '충',
 '가',
 '후',
 '8',
 '쉽',
 '세',
 '야',
 '언',
 '병',
 '님',
 '間',
 '누',
 '큰',
 '두',
 '행',
 '척',
 '치',
 'P',
 '노',
 '섭',
 '신',
 '논',
 '스',
 'U',
 '터',
 '@',
 '넓',
 '국',
 '느',
 '침',
 '효',
 '「',
 'L',
 '타',
 '별',
 '더',
 '런',
 '녀',
 '없',
 '완',
 '참',
 '찰',
 '검',
 'm',
 '렴',
 '협',
 '권',
 '습',
 '폭',
 '향',
 '볼',
 ':',
 '고',
 '期',
 '한',
 '지',
 '촉',
 '헌',
 '년',
 '김',
 '잃',
 '범',
 '격',
 '삭',
 '성',
 '｣',
 '務',
 '원',
 '준',
 '◦',
 '하',
 '파',
 '같',
 '높',
 '에',
 '허',
 '훈',
 '최',
 '진',
 '단',
 '비',
 '①',
 '걸',
 'z',
 'p',
 '제',
 '條',
 '교',
 '면',
 '윤',
 '⋅',
 '[',
 '②',
 '늘',
 '선',
 '방',
 '많',
 '현',
 ',',
 '事',
 '리',
 '령',
 '못',
 'e',
 '포',
 '핌',
 '이',
 '｢',
 '權',
 '셋',
 '편',
 '우',
 '율',
 '승',
 '흉',
 '도',
 '질',
 '니',
 '된',
 '따',
 '매',
 '헬',
 '특',
 'b',
 '통',
 '응',
 '내',
 '출',
 '7',
 '”',
 '들',
 '호',
 '나',
 '정'

In [37]:
from collections import defaultdict
from konlpy.tag import Kkma

ma = Kkma().morphs

def getDocReprByDefaultDict(lexicon):
    docRepr = defaultdict(lambda: defaultdict(int))
    
    for docName in kobill.fileids():
        document = kobill.open(docName).read()
 
        for token in document.split(): 
            for morpheme in ma(token):
                docRepr[docName][morpheme] += 1
            
    return docRepr

In [38]:
txt = getLexiconBySet()
DTM = getDocReprByDefaultDict(txt)
DTM

defaultdict(<function __main__.getDocReprByDefaultDict.<locals>.<lambda>()>,
            {'1809896.txt': defaultdict(int,
                         {'행정': 38,
                          '절차법': 3,
                          '일부': 4,
                          '개정': 5,
                          '법률안': 2,
                          '(': 52,
                          '유': 3,
                          '선호': 2,
                          '의원': 2,
                          '대표': 1,
                          '발의': 2,
                          ')': 52,
                          '의': 181,
                          '안': 24,
                          '벌': 1,
                          'ㄴ': 118,
                          '호': 13,
                          '9896': 1,
                          '연월일': 1,
                          ':': 2,
                          '2010': 1,
                          '.': 141,
                          '11': 4,
                          '15': 4,
                          '발':

In [39]:
# invertedDocument (역문헌구조, 어휘)
def convertInvertedDocument(DTM):
    TDM = defaultdict(lambda: defaultdict(int))
    
    for fileName, termList in DTM.items():  
        maxFreq = max(termList.values())
        for term, freq in termList.items():
            TDM[term][fileName] = maxTF(0,freq,maxFreq)
            
    return TDM

In [40]:
# 빈도로 나타내기 위해 기존의 getDocReprByDefaultDict의= 를 +=로 변경 
TDM = convertInvertedDocument(DTM)
TDM

defaultdict(<function __main__.convertInvertedDocument.<locals>.<lambda>()>,
            {'행정': defaultdict(int,
                         {'1809896.txt': 0.16379310344827586,
                          '1809890.txt': 0.016260162601626018,
                          '1809891.txt': 0.01680672268907563,
                          '1809893.txt': 0.021739130434782608,
                          '1809892.txt': 0.01,
                          '1809899.txt': 0.004651162790697674}),
             '절차법': defaultdict(int, {'1809896.txt': 0.01293103448275862}),
             '일부': defaultdict(int,
                         {'1809896.txt': 0.017241379310344827,
                          '1809895.txt': 0.06976744186046512,
                          '1809894.txt': 0.10344827586206896,
                          '1809890.txt': 0.032520325203252036,
                          '1809891.txt': 0.03361344537815126,
                          '1809893.txt': 0.043478260869565216,
                          '1809892.txt

In [41]:
# term-document -> term weight
# defaultdict 를 써서 key 걱정을 안해도 된다. 
N = len(DTM)

def TDM2TWM(TDM):
    TWM = defaultdict(lambda: defaultdict(float))
    DVL = defaultdict(float)
    
    for term, tfList in TDM.items():
        df = len(tfList)
        idf = rawIdf(df,N)
        for fileName, tf in tfList.items():
            TWM[term][fileName] = tf * idf
            DVL[fileName] += TWM[term][fileName]  ** 2
            
    return TWM, DVL
    

In [42]:
TWM,DVL = TDM2TWM(TDM)

In [43]:
#TWM

In [44]:
DVL

defaultdict(float,
            {'1809896.txt': 0.23728598660065164,
             '1809890.txt': 0.08891068576179656,
             '1809891.txt': 0.09407896411798816,
             '1809893.txt': 0.14887973976614557,
             '1809892.txt': 0.056003860773669124,
             '1809899.txt': 0.08503881498928928,
             '1809895.txt': 0.3553986340082429,
             '1809894.txt': 0.5467267027461256,
             '1809897.txt': 0.8414240759234843,
             '1809898.txt': 0.5833111894640503})

In [45]:
query = "국방의 의무와 보편적 교육에 대한 법안"

In [46]:
queryRepr = defaultdict(int) # 빈도를 갖는

for token in query.split(): 
    for morpheme in ma(token):
        queryRepr[morpheme] += 1
        
queryWeight = defaultdict(float)
maxFreq = max(queryRepr.values())

for token, freq in queryRepr.items():
    if token in TWM.keys():
        tf = maxTF(0.5,freq,maxFreq)
        df = len(TWM[token])
        idf = rawIdf(df,N)
        queryWeight[token] = tf * idf

In [47]:
queryWeight

defaultdict(float,
            {'국방': 0.6989700043360189,
             '의': 0.0,
             '의무': 0.6989700043360189,
             '와': 0.04575749056067514,
             '교육': 0.09691001300805642,
             '에': 0.0,
             '대하': 0.0,
             'ㄴ': 0.0,
             '법안': 0.3979400086720376})

In [48]:
from math import sqrt

candidateList = defaultdict(float)

for token, weight in queryWeight.items():
    for fileName, tfidf in TWM[token].items():
        print(" {0} : {1} = {2} * {3}".format(
        token, fileName,weight,tfidf))
        candidateList[fileName] += innerProduct(weight, tfidf)
        
for fileName, sumProduct in candidateList.items():
    candidateList[fileName] /= sqrt(DVL[fileName])

 국방 : 1809897.txt = 0.6989700043360189 * 0.02184281263550059
 국방 : 1809898.txt = 0.6989700043360189 * 0.00998528577622884
 의 : 1809896.txt = 0.0 * 0.0
 의 : 1809897.txt = 0.0 * 0.0
 의 : 1809895.txt = 0.0 * 0.0
 의 : 1809894.txt = 0.0 * 0.0
 의 : 1809890.txt = 0.0 * 0.0
 의 : 1809891.txt = 0.0 * 0.0
 의 : 1809893.txt = 0.0 * 0.0
 의 : 1809892.txt = 0.0 * 0.0
 의 : 1809899.txt = 0.0 * 0.0
 의 : 1809898.txt = 0.0 * 0.0
 의무 : 1809896.txt = 0.6989700043360189 * 0.0060256034856553346
 의무 : 1809899.txt = 0.6989700043360189 * 0.0016255116379907415
 와 : 1809896.txt = 0.04575749056067514 * 0.001972305627615308
 와 : 1809897.txt = 0.04575749056067514 * 0.0028598431600421964
 와 : 1809895.txt = 0.04575749056067514 * 0.0031923830623726843
 와 : 1809890.txt = 0.04575749056067514 * 0.0011160363551384182
 와 : 1809891.txt = 0.04575749056067514 * 0.0011535501822018943
 와 : 1809893.txt = 0.04575749056067514 * 0.0014920920835002763
 와 : 1809892.txt = 0.04575749056067514 * 0.0006863623584101271
 와 : 1809899.txt = 0.0

In [49]:
from nltk.tokenize import sent_tokenize
K=5

resultList = sorted(candidateList.items(), key = lambda x:x[1], reverse=True)
for i,(fileName, similarity) in enumerate(resultList):
    if i < K:
        print(" Rank:{0} / Document:{1} / Similarity:{2:.4f}".format((i+1),fileName,similarity))
        content = kobill.open(fileName).read()
        print(sent_tokenize(content)[:5])

 Rank:1 / Document:1809897.txt / Similarity:0.0189
['국군부대의 아랍에미리트(UAE)군 교육훈련 지원 등에 \n관한 파견 동의안\n\n의안\n                                                                  제출연월일 :  2010.', '11.', '15.', '9897\n번호\n                                                                        제  출  자 :  정         부\n\n제안이유\n\n    가.  UAE측 요청과 협의에 따라,  국익창출과  다양한 지역에서의 \n우리 특전부대 임무수행능력 향상 등을 목적으로  국군부대를 \nUAE에 파견하려는 것임.', '나.']
 Rank:2 / Document:1809898.txt / Similarity:0.0092
['국군부대의 소말리아 해역 파견연장 동의안\n\n의안\n                                                                  제출연월일 :  2010.', '11.', '15.', '9898\n번호\n                                                                        제  출  자 :  정         부\n\n제안이유\n\n      소말리아 아덴만 해역에 파견된 국군부대 ( 청해부대 )의 파견기간이 \n2010년 12월 31일 종료될 예정이나,  다음 이유로 파견기간을 연장\n하고자 함.', '첫째,  소말리아 해적활동으로 우리 선박의 안전이 위협을 받고 있고,\n\n      둘째,  청해부대가  성공적으로 임무를 수행하여 우리 국익보호 및  국위\n선양에 기여하고 있으며,   \n\n      셋째,  국내외 관계기관에서 파견연장을 요청하고 있음.']
 Rank:3 / Document:1809896.txt / Simi

In [ ]:
type(content)